## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather description
0,0,Bulawayo,-20.1500,28.5833,55.44,68,62,4.90,broken clouds
1,1,Poum,-20.2333,164.0167,79.23,71,100,9.28,overcast clouds
2,2,Upernavik,72.7868,-56.1549,2.28,72,100,5.50,overcast clouds
3,3,Chuy,-33.6971,-53.4616,61.86,76,98,6.58,overcast clouds
4,4,Hithadhoo,-0.6000,73.0833,83.79,76,100,19.95,overcast clouds
5,5,Kitsuki,33.4167,131.6167,57.20,67,75,11.50,broken clouds
6,6,Khatanga,71.9667,102.5000,-27.22,95,36,0.51,scattered clouds
7,7,Sorland,67.6670,12.6934,39.15,74,100,21.32,overcast clouds
8,8,Ribeira Grande,38.5167,-28.7000,57.81,66,0,10.71,clear sky
9,9,Barrow,71.2906,-156.7887,-4.00,59,1,23.02,clear sky


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 120


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
clean_df.head(10)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather description
1,1,Poum,-20.2333,164.0167,79.23,71,100,9.28,overcast clouds
3,3,Chuy,-33.6971,-53.4616,61.86,76,98,6.58,overcast clouds
4,4,Hithadhoo,-0.6000,73.0833,83.79,76,100,19.95,overcast clouds
10,10,Laguna,38.4210,-121.4238,80.60,17,1,4.61,clear sky
11,11,Kota Bahru,6.1333,102.2386,84.20,84,20,3.44,few clouds
13,13,Arraial Do Cabo,-22.9661,-42.0278,74.48,77,45,16.75,scattered clouds
15,15,Busselton,-33.6500,115.3333,69.01,82,100,1.99,overcast clouds
16,16,Port Elizabeth,-33.9180,25.5701,64.40,77,0,10.36,clear sky
17,17,Pisco,-13.7000,-76.2167,68.00,77,0,17.27,clear sky
18,18,Saint-Philippe,-21.3585,55.7679,78.80,78,0,9.22,light rain


In [9]:
# 4a. Determine if there are any empty rows.
clean_df.count()


City_ID                376
City                   376
Lat                    376
Lng                    376
Max Temp               376
Humidity               376
Cloudiness             376
Wind Speed             376
Weather description    376
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)].dropna()
clean_df.count()


City_ID                376
City                   376
Lat                    376
Lng                    376
Max Temp               376
Humidity               376
Cloudiness             376
Wind Speed             376
Weather description    376
dtype: int64

In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Max Temp", "Weather description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Max Temp,Weather description,Lat,Lng,Hotel Name
1,Poum,79.23,overcast clouds,-20.2333,164.0167,
3,Chuy,61.86,overcast clouds,-33.6971,-53.4616,
4,Hithadhoo,83.79,overcast clouds,-0.6000,73.0833,
10,Laguna,80.60,clear sky,38.4210,-121.4238,
11,Kota Bahru,84.20,few clouds,6.1333,102.2386,
13,Arraial Do Cabo,74.48,scattered clouds,-22.9661,-42.0278,
15,Busselton,69.01,overcast clouds,-33.6500,115.3333,
16,Port Elizabeth,64.40,clear sky,-33.9180,25.5701,
17,Pisco,68.00,clear sky,-13.7000,-76.2167,
18,Saint-Philippe,78.80,light rain,-21.3585,55.7679,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        print(index)
    except (IndexError):
        print("Hotel not found... skipping.")
        

1
3
4
10
11
13
15
16
17
18
19
20
Hotel not found... skipping.
23
24
25
26
29
30
31
32
33
34
35
37
38
39
40
41
42
43
46
49
50
51
52
53
56
57
58
61
63
65
66
68
69
71
73
74
75
78
79
80
82
84
88
Hotel not found... skipping.
90
91
92
93
94
95
98
99
100
101
103
104
105
106
109
111
112
113
114
115
116
118
121
124
125
130
136
140
141
144
145
147
148
Hotel not found... skipping.
152
154
155
157
158
165
166
168
171
174
175
176
177
178
182
184
185
186
188
189
190
191
193
194
198
201
203
204
206
208
209
211
212
213
214
215
218
220
221
223
224
225
227
229
232
233
236
242
Hotel not found... skipping.
244
246
248
249
250
251
252
255
256
257
258
259
264
Hotel not found... skipping.
266
269
270
271
275
276
277
279
281
282
284
Hotel not found... skipping.
287
288
290
293
296
301
306
311
313
314
315
316
318
319
321
322
323
325
328
329
Hotel not found... skipping.
331
332
335
336
340
341
342
344
345
346
348
349
351
353
355
356
360
361
362
363
Hotel not found... skipping.
366
367
368
369
373
374
376
Hotel 

In [13]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.dropna()
hotel_df

,City,Max Temp,Weather description,Lat,Lng,Hotel Name
1,Poum,79.23,overcast clouds,-20.2333,164.0167,Gîte kajeon
3,Chuy,61.86,overcast clouds,-33.6971,-53.4616,Nuevo Hotel Plaza
4,Hithadhoo,83.79,overcast clouds,-0.6000,73.0833,Scoop Guest House
10,Laguna,80.60,clear sky,38.4210,-121.4238,Holiday Inn Express & Suites Elk Grove Central...
11,Kota Bahru,84.20,few clouds,6.1333,102.2386,Grand Riverview Hotel
...,...,...,...,...,...,...
671,Sechura,73.18,scattered clouds,-5.5569,-80.8222,Hospedaje Costa Bella
673,Phayuha Khiri,84.99,broken clouds,15.4553,100.1353,Country Lake Nature Lodge
674,Vanimo,87.35,broken clouds,-2.6741,141.3028,Vahmoneh Lodge Vanimo
675,Gladstone,78.80,light rain,-23.8500,151.2500,Highpoint International


In [14]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Weather description</dt><dd>{Weather description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))